# CSV to create issues and add to Project Board

Complete system with git API calls and bulk creation of issues.

In [78]:
# Setup
import pandas as pd
from cmipld.utils.git.projects import *

ORG = "WCRP-CMIP"
REPO = "CVsTT-Project-Planning"

ghutils = GitHubUtils()
teams  = TeamManager()
issue_mgr = IssueManager()
label_mgr = LabelManager()
milestone_mgr = MilestoneManager()
project_mgr = ProjectManager()

print("✅ Ready - all APIs initialized.")

✅ Ready - all APIs initialized.


In [37]:
# project_mgr.list_all_projects(ORG)
pid = 'PVT_kwDOATRwu84A60Fm'


In [4]:
attributes = pd.read_csv('global_attributes_tasks.csv')
attributes.head()

,title,start_date,end_date,milestone,labels,assignees,content
0,Global_attributes,2025-05-22,2025-06-05,"DRS, GlobalAttribute","DRS,GlobalAttribute,CVsTT,DataModeler",CVsTT,# Global Attribute Review\n\n## Status\n- [ ] ...
1,variableRootDD,2025-05-29,2025-06-12,"DRS, GlobalAttribute","DRS,GlobalAttribute,CVsTT,DR",CVsTT,# Global Attribute Review\n\n## Status\n- [ ] ...
2,horizontalLabelDD,2025-05-29,2025-06-12,"DRS, GlobalAttribute","DRS,GlobalAttribute,CVsTT,KarlDocument",CVsTT,# Global Attribute Review\n\n## Status\n- [ ] ...
3,verticalDD,2025-05-29,2025-06-12,"DRS, GlobalAttribute","DRS,GlobalAttribute,CVsTT,KarlDocument",CVsTT,# Global Attribute Review\n\n## Status\n- [ ] ...
4,temporalLabelDD,2025-05-29,2025-06-12,"DRS, GlobalAttribute","DRS,GlobalAttribute,CVsTT,KarlDocument",CVsTT,# Global Attribute Review\n\n## Status\n- [ ] ...


In [8]:
other = pd.read_csv('framework_tasks.csv')
other['assignees']=''
other.head()

,title,start_date,end_date,milestone,labels,assignees,content
0,JSONLD rename homodeniguty,2025-05-22,2025-07-03,Framework,"Framework,CVsTT",,# Framework Task Review\n\n## Status\n- [ ] St...
1,Match CV names with DD names,2025-05-22,2025-11-06,Framework,"Framework,CVsTT",,# Framework Task Review\n\n## Status\n- [x] On...
2,README documentation,2025-05-22,2025-07-03,Framework,"Framework,CVsTT,Documentation",,# Framework Task Review\n\n## Status\n- [ ] St...
3,Get Scientific members to populate field descr...,2025-06-19,2025-11-06,Framework,"Framework,CVsTT,Documentation",,# Framework Task Review\n\n## Status\n- [ ] Ne...
4,Symbiosis of esgvoc and cmip-ld,2025-05-22,2025-06-19,Framework,"Framework,CVsTT,Integration",,# Framework Task Review\n\n## Status\n- [x] On...


In [ ]:
def create_new(data):
    labels = set()
    milestones = set()
    assign = set()

    # add all the labels and milestones from the attributes
    for i, row in data.iterrows():
            
            labels = labels.union(set(row['labels'].split(',')))
            milestones = milestones.union(set(row['milestone'].split(',')))
            assign = assign.union(set(row['assignees'].split(',')))
            
    label_mgr.ensure_labels_exist(ORG, REPO, [{"name":i,"description":i} for i in labels])
    [milestone_mgr.ensure_milestone_exists(ORG, REPO, j) for j in [{"title":i,"description":i} for i in milestones]]
    [teams.create_child_team(ORG, 'CVsTT', i, description=None) for i in assign if i != '']




    idata=[]
    for i, row in data.iterrows():
            idata .append({
            "title": row['title'],
            "content": f'{row["content"]}',
            "labels": row['labels'].split(','),
            "milestone": row['milestone'].split(',')[0],
            "start_date": row['start_date'],
            "end_date": row['end_date'],
            "teams": row['assignees'].split(','),
        })

    # Create all roadmap issues with project integration
    created, failed = issue_mgr.create_issues_bulk(
        ORG, 
        REPO, 
        idata,
        add_to_project_id=pid
    )

        
    for i in created:
        t = list(filter(lambda x: x['title'] == i['title'],idata))[0]
        for t2 in t['teams']:
            teams.assign_issue_to_team(i['url'], ORG,t2)






In [ ]:
# create_new(attributes)


🏷️ Ensuring labels exist...

🏷️ Checking if label 'New' exists...
🔄 Running: gh api /repos/WCRP-CMIP/CVsTT-Project-Planning/labels/New
Return code: 0
Stdout: {
  "id": 8736415893,
  "node_id": "LA_kwDOO23ghc8AAAACCLsglQ",
  "url": "https://api.github.com/repos/WCRP-CMIP/CVsTT-Project-Planning/labels/New",
  "name": "New",
  "color": "FFFFFF",
  "default": ...
✅ Label 'New' exists

🏷️ Checking if label 'GlobalAttribute' exists...
🔄 Running: gh api /repos/WCRP-CMIP/CVsTT-Project-Planning/labels/GlobalAttribute
Return code: 0
Stdout: {
  "id": 8736415900,
  "node_id": "LA_kwDOO23ghc8AAAACCLsgnA",
  "url": "https://api.github.com/repos/WCRP-CMIP/CVsTT-Project-Planning/labels/GlobalAttribute",
  "name": "GlobalAttribute",
  "color":...
✅ Label 'GlobalAttribute' exists

🏷️ Checking if label 'Input4Mips' exists...
🔄 Running: gh api /repos/WCRP-CMIP/CVsTT-Project-Planning/labels/Input4Mips
Return code: 0
Stdout: {
  "id": 8736415916,
  "node_id": "LA_kwDOO23ghc8AAAACCLsgrA",
  "url": "https:/

In [57]:
# create_new(other) 

In [46]:
help(
issue_mgr.create_issues_bulk )

Help on method create_issues_bulk in module cmipld.utils.git.projects.issues:

create_issues_bulk(repo_owner, repo_name, issues_data, add_to_project_id=None) method of cmipld.utils.git.projects.issues.IssueManager instance
    Create multiple issues from structured data with optional project integration



In [81]:
import os,json
urls = [i['url'] for i in json.loads(ghutils.run_gh_cmd('issue list --repo WCRP-CMIP/CVsTT-Project-Planning --limit 1000 --json url'.split()))]

teams.create_child_team(ORG, 'CVsTT', 'CVs-Framework', description='All things technical to do with the CV infrastructure and framework.',)
for url in urls:
    teams.assign_issue_to_team(url, ORG, 'CVs-Framework')

🔄 Running: gh issue list --repo WCRP-CMIP/CVsTT-Project-Planning --limit 1000 --json url
Return code: 0
Stdout: [
  {
    "url": "https://github.com/WCRP-CMIP/CVsTT-Project-Planning/issues/9"
  },
  {
    "url": "https://github.com/WCRP-CMIP/CVsTT-Project-Planning/issues/8"
  },
  {
    "url": "https://github.c...

➕ Creating child team 'CVs-Framework' under 'CVsTT'...
🔄 Running: gh api /orgs/WCRP-CMIP/teams/CVsTT
Return code: 0
Stdout: {
  "name": "CVsTT",
  "id": 12650392,
  "node_id": "T_kwDOATRwu84AwQeY",
  "slug": "cvstt",
  "description": "CVs task team",
  "privacy": "closed",
  "notification_setting": "notifications_enabled",...

➕ Creating team 'CVs-Framework'...
🔄 Running: gh api /orgs/WCRP-CMIP/teams --method POST --input -
Return code: 1
Stdout: {
  "message": "Validation Failed",
  "errors": [
    {
      "resource": "Team",
      "code": "unprocessable",
      "field": "data",
      "message": "Name must be unique for this org"
    }
  ],
 ...
Stderr: gh: Validation Faile